### Дивертисмент: Диалог Клиента и БД

Клиент 🥭: Дай мне список возможных конструкций  

БД 💻: Отдает **СПИСОК КОНСТРУКЦИЙ**

Клиент 🥭: Дай мне список возможных материалов для **ЭТОЙ КОНСТРУКЦИИ**

БД 💻: **СПИСОК МАТЕРИАЛОВ** для **ЭТОЙ КОНСТРУКЦИИ**

Клиент 🥭: Дай мне список **ВОЗМОЖНЫХ ДЕФФЕКТОВ** для **ЭТОЙ КОНСТРУКЦИИ** из **ЭТОГО МАТЕРИАЛА**

БД 💻: **СПИСОК ВОЗМОЖНЫХ ДЕФФЕКТОВ**

Клиент 🥭: Я обнаружил **ЭТОТ ДЕФФЕКТ** дай мне все возможные данные о нем

БД 💻: **ЭТИ ДАННЫЕ** для **ЭТОГО ДЕФФЕКТА**

Клиент 🥭: Оценивает **ЭТОТ ДЕФФЕКТ** на основе **ЭТИХ ДАННЫХ**

## Создание БД

In [ ]:
import sqlite3

# Создаем подключение к базе данных SQLite
with sqlite3.connect('defect_db.db') as conn:
  cursor = conn.cursor()

  # Создаем таблицы
  cursor.execute('''CREATE TABLE IF NOT EXISTS constructions (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      name TEXT NOT NULL
  )''')

  cursor.execute('''CREATE TABLE IF NOT EXISTS materials (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      construction_id INTEGER,
      name TEXT NOT NULL,
      FOREIGN KEY(construction_id) REFERENCES constructions(id)
  )''')

  cursor.execute('''CREATE TABLE IF NOT EXISTS defects (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      construction_id INTEGER,
      material_id INTEGER,
      name TEXT NOT NULL,
      FOREIGN KEY(construction_id) REFERENCES constructions(id),
      FOREIGN KEY(material_id) REFERENCES materials(id)
  )''')

  # От поля options_JSON определяется view пользоватля
  # Либо будет селектор со значениями
  # Либо поля с вводами и некоторой валидацией (на это забъем на начальных этапах)
  #
  """
  {
    type: selectable,
    values:  [скучно, нудно, все вымерли]
  }

  {
    type: field,
    values:  [
      {
        name: "ширина трещины",
        type: int
        },
      {
        name: "глубина трещины",
        type: int
        }
      ]
  }
  """
  #
  #
  cursor.execute('''CREATE TABLE IF NOT EXISTS parameters (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      defect_id INTEGER NOT NULL,
      options_JSON TEXT,
      FOREIGN KEY(defect_id) REFERENCES defects(id)
  )''')

## Предпологаемые запросы к БД
Пользователь никак не взаимадействует с БД
Бизнес логика выстраивается на стороне клиента
Достаточные для вынесения вердикта данные лежат в json для каждого конкретного дефекта

In [ ]:
def get_avaliable_constructions():
    cursor.execute('SELECT * FROM constructions')
    return cursor.fetchall()

def get_avaliable_materials(construction_id):
    cursor.execute('SELECT id, name FROM materials WHERE construction_id = ?', (construction_id,))
    return cursor.fetchall()


def get_avaliable_defects(construction_id, material_id):
    cursor.execute('SELECT id, name FROM defects WHERE construction_id = ? AND material_id = ?', (construction_id, material_id))
    return cursor.fetchall()

def get_parameters(defect_id):
    cursor.execute('SELECT options_JSON FROM parameters WHERE defect_id = ?', (defect_id,))
    return cursor.fetchone()



## Предпологоаемая модель json для деффекта

```json
  {
  "name": "Трещина",
  "measure": "мм",
  "оценка": {
      1: [0, 0.1],
      2: [0.1, 0.3],
      3: [0.3, 0.5],
      4: [0.5, 1],
      5: [1, 1000000]
    },
  }
```

```json
  {
  "name": "Трещина",
  "measure": "мм",
  "оценка": {
      1: [0, 0.1],
      2: [0.1, 0.3],
      3: [0.3, 0.5],
      4: [0.5, 1],
      5: [1, 1000000]
    },
  }

Коррозия арматуры,"1. Нет, 2. До 10% площади, 3. До 15% площади, 4. Оголение арматуры, 5. Коррозия более 15%",1. Нет — 1
,,2. До 10% площади — 2
,,3. До 15% площади — 3
,,4. Оголение арматуры — 4
,,5. Коррозия более 15% — 5
```


## Наполнение БД

In [ ]:
with sqlite3.connect('defect_db.db') as conn:
  cursor = conn.cursor()
  cursor.execute("INSERT INTO constructions (id, name) VALUES (?, ?)", [0, "Неизвестная"])
  cursor.fetchall()
  conn.commit()

In [ ]:
with sqlite3.connect('defect_db.db') as conn:
  cursor = conn.cursor()
  cursor.executemany('''INSERT INTO materials (construction_id, name) VALUES (?, ?);''', [
  (0, "Стальная конструкция"),
  (0, "Железобетонная конструкция"),
  (0, "Железобетонная конструкция"),
  (0, "Каменная конструкция"),
  (0, "Деревянная конструкция")])
  cursor.fetchall()
  conn.commit()


In [ ]:
with sqlite3.connect('defect_db.db') as conn:
  cursor = conn.cursor()
  cursor.execute("SELECT * FROM materials")
  cursor.fetchall()


In [ ]:
with sqlite3.connect('defect_db.db') as conn:
  cursor = conn.cursor()
  cursor.executemany("""
  INSERT INTO defects (name, material_id, construction_id)
  VALUES (?,
          (SELECT id FROM materials where name = ? AND construction_id = 0),
          0)   ;

  """,
        [("Признаки силовых воздействий", "Стальная конструкция"),
         ("Коррозия", "Стальная конструкция"),
        ("Погнутости от ударов", "Стальная конструкция"),
        ("Трещины в бетоне", "Железобетонная конструкция"),
        ("Коррозия арматуры", "Железобетонная конструкция"),
        ("Прочность бетона", "Железобетонная конструкция"),
        ("Трещины в кладке", "Каменная конструкция"),
        ("Выветривание и отслоение", "Каменная конструкция"),
        ("Трещины в растянутой зоне", "Железобетонная конструкция"),
        ("Коррозия арматуры", "Железобетонная конструкция"),
        ("Прогиб изгибаемых элементов", "Деревянная конструкция"),
        ("Поражение гнилью", "Деревянная конструкция"),
        ("Наклон и выпучивание стен", "Каменная конструкция"),
         ]
  )
  cursor.fetchall();
  conn.commit()



In [ ]:
with sqlite3.connect('defect_db.db') as conn:
  cursor = conn.cursor()
  cursor.execute("SELECT * FROM defects")
  print(cursor.fetchall())


[(1, 0, 1, 'Признаки силовых воздействий'), (2, 0, 1, 'Коррозия'), (3, 0, 1, 'Погнутости от ударов'), (4, 0, 2, 'Трещины в бетоне'), (5, 0, 2, 'Коррозия арматуры'), (6, 0, 2, 'Прочность бетона'), (7, 0, 4, 'Трещины в кладке'), (8, 0, 4, 'Выветривание и отслоение'), (9, 0, 2, 'Трещины в растянутой зоне'), (10, 0, 2, 'Коррозия арматуры'), (11, 0, 5, 'Прогиб изгибаемых элементов'), (12, 0, 5, 'Поражение гнилью'), (13, 0, 4, 'Наклон и выпучивание стен')]


### СУЩИЙ КОШМАР И МРАКОБЕСИЕ

In [ ]:
import json

In [ ]:
# Данные для заполнения таблицы
parameters_data = [
    {
        "defect_id": 1,
        "options_JSON": json.dumps({
            "name": "Признаки силовых воздействий",
            "measure": "отношение пролета",
            "оценка": {
                1: [0, 0],
                2: [0, 0.0067],
                3: [0.0067, 0.0133],
                4: [0.0133, 0.02],
                5: [0.02, 1000000]
            }
        })
    },
    {
        "defect_id": 2,
        "options_JSON": json.dumps({
            "name": "Коррозия",
            "measure": "% сечения",
            "оценка": {
                1: [0, 0],
                2: [0, 5],
                3: [5, 15],
                4: [15, 25],
                5: [25, 1000000]
            }
        })
    },
    {
        "defect_id": 3,
        "options_JSON": json.dumps({
            "name": "Погнутости от ударов",
            "measure": "% ослабления",
            "оценка": {
                1: [0, 0],
                2: [0, 5],
                3: [5, 15],
                4: [15, 25],
                5: [25, 1000000]
            }
        })
    },
    {
        "defect_id": 4,
        "options_JSON": json.dumps({
            "name": "Трещины в бетоне",
            "measure": "мм",
            "оценка": {
                1: [0, 0.1],
                2: [0.1, 0.3],
                3: [0.3, 0.5],
                4: [0.5, 1],
                5: [1, 1000000]
            }
        })
    },
    {
        "defect_id": 5,
        "options_JSON": json.dumps({
            "name": "Коррозия арматуры",
            "measure": "% площади",
            "оценка": {
                1: [0, 0],
                2: [0, 10],
                3: [10, 15],
                4: [15, 100],
                5: [100, 1000000]
            }
        })
    },
    {
        "defect_id": 6,
        "options_JSON": json.dumps({
            "name": "Прочность бетона",
            "measure": "% прочности",
            "оценка": {
                1: [100, 100],
                2: [80, 100],
                3: [70, 80],
                4: [0, 70],
                5: [0, 30]
            }
        })
    },
    {
        "defect_id": 7,
        "options_JSON": json.dumps({
            "name": "Трещины в кладке",
            "measure": "ряды",
            "оценка": {
                1: [0, 0],
                2: [0, 2],
                3: [2, 4],
                4: [4, 1000000],
                5: [1000000, float("inf")]
            }
        })
    },
    {
        "defect_id": 8,
        "options_JSON": json.dumps({
            "name": "Выветривание и отслоение",
            "measure": "% толщины",
            "оценка": {
                1: [0, 0],
                2: [0, 1],
                3: [1, 15],
                4: [15, 25],
                5: [25, 40]
            }
        })
    },
    {
        "defect_id": 9,
        "options_JSON": json.dumps({
            "name": "Трещины в растянутой зоне",
            "measure": "мм",
            "оценка": {
                1: [0, 0.1],
                2: [0.1, 0.3],
                3: [0.3, 0.5],
                4: [0.5, 1],
                5: [1, float("inf")]
            }
        })
    },
    {
        "defect_id": 10,
        "options_JSON": json.dumps({
            "name": "Коррозия арматуры",
            "measure": "% сечения",
            "оценка": {
                1: [0, 0],
                2: [0, 10],
                3: [10, 15],
                4: [15, 25],
                5: [25, float("inf")]
            }
        })
    },
    {
        "defect_id": 11,
        "options_JSON": json.dumps({
            "name": "Прогиб изгибаемых элементов",
            "measure": "отношение пролета",
            "оценка": {
                1: [0, 0],
                2: [0, 1],  # Ослабление креплений — условная оценка
                3: [0, 0.02],  # Превышение СНиП
                4: [0.02, 0.0133],
                5: [0.0133, float("inf")]
            }
        })
    },
    {
        "defect_id": 12,
        "options_JSON": json.dumps({
            "name": "Поражение гнилью",
            "measure": "%",
            "оценка": {
                1: [0, 0],
                2: [0, 15],
                3: [15, 25],
                4: [25, 100],
                5: [100, float("inf")]
            }
        })
    },
    {
        "defect_id": 13,
        "options_JSON": json.dumps({
            "name": "Наклон и выпучивание стен",
            "measure": "отношение толщины",
            "оценка": {
                1: [0, 0],
                2: [0, 0.167],  # 1/6
                3: [0.167, 0.15],  # На глубину до 15%
                4: [0.15, 0.333],  # До 1/3 толщины
                5: [0.333, float("inf")]
            }
        })
    }
]


### Наполнение Jsonamи

In [ ]:
import sqlite3
import json

# Подключение к базе данных SQLite
conn = sqlite3.connect('defect_db.db')
cursor = conn.cursor()



# Вставка данных в таблицу parameters
for param in parameters_data:
    cursor.execute('INSERT INTO parameters (defect_id, options_JSON) VALUES (?, ?)',
                   (param['defect_id'], param['options_JSON']))

# Сохраняем изменения и закрываем соединение
conn.commit()
conn.close()

print("Данные успешно добавлены в таблицу parameters.")


Данные успешно добавлены в таблицу parameters.


## Пример взаимодействия пользователя с БД

Данный пример является лишь примером
На практике данные взаимодействия будут происходить между клиентским приложением и сервером. Рассмотрим случай в котором клиент выбирает *Неизвестный объект, который является деревянной конструкцией*

Пользователь выбирает Объект
Посылается запрос к БД и предоставляется список доступных Объектов

In [ ]:
conn = sqlite3.connect('defect_db.db')
cursor = conn.cursor()

constructions = get_avaliable_constructions()
constructions


[(0, 'Неизвестная')]

Выбрав из выпадающего меню конструкцию пользователь выбирает более конкретное значение

In [ ]:
materials = get_avaliable_materials(constructions[0][0])
materials

[(1, 'Стальная конструкция'),
 (2, 'Железобетонная конструкция'),
 (3, 'Железобетонная конструкция'),
 (4, 'Каменная конструкция'),
 (5, 'Деревянная конструкция')]

Пользователь выбирает доступную конструкцию из ему выводится список деффектов

In [ ]:
defects = get_avaliable_defects(constructions[0][0], materials[4][0])
defects

[(11, 'Прогиб изгибаемых элементов'), (12, 'Поражение гнилью')]

И далее наконец-то узнает какие параметры нужно ему ввести

In [ ]:
params = (get_parameters(11), get_parameters(12))
(json.loads(params[0][0]), json.loads(params[1][0]))


({'name': 'Прогиб изгибаемых элементов',
  'measure': 'отношение пролета',
  'оценка': {'1': [0, 0],
   '2': [0, 1],
   '3': [0, 0.02],
   '4': [0.02, 0.0133],
   '5': [0.0133, inf]}},
 {'name': 'Поражение гнилью',
  'measure': '%',
  'оценка': {'1': [0, 0],
   '2': [0, 15],
   '3': [15, 25],
   '4': [25, 100],
   '5': [100, inf]}})

Пользователю выводится только название и единица измерения,

После чего при вводе значения клиентское приложение самостоятельно высчитывает оценку и выводит результат